In [1]:
using Gync, Vega

In [2]:
iters = 10000
rho = 0.1
sigmay0 = 3
sigmap  = 2

config = ModelConfig(Subject(:lausanne, 1), rho, sigmay0, sigmap)

sim = runsim(config, iters, relprop=0.1);

MCMC Simulation of 10000 Iterations x 1 Chain...

Chain 1:   0% [1:22:25 of 1:22:30 remaining]
Chain 1:  10% [0:04:58 of 0:05:31 remaining]
Chain 1:  20% [0:04:06 of 0:05:08 remaining]
Chain 1:  30% [0:03:28 of 0:04:57 remaining]
Chain 1:  40% [0:02:57 of 0:04:55 remaining]
Chain 1:  50% [0:02:27 of 0:04:53 remaining]
Chain 1:  60% [0:01:57 of 0:04:52 remaining]
Chain 1:  70% [0:01:28 of 0:04:52 remaining]
Chain 1:  80% [0:00:58 of 0:04:51 remaining]
Chain 1:  90% [0:00:29 of 0:04:50 remaining]
Chain 1: 100% [0:00:00 of 0:04:50 remaining]



In [3]:
using Vega

import Vega: VegaData

# Construct VegaData from a given Markov Chain
VegaData(chains::Mamba.AbstractChains, args...) = VegaData(squeeze(chains.value, 3), chains.names, name="samples")

function VegaData{S<:AbstractString}(m::Matrix, colnames::Vector{S}; 
    name::AbstractString = string("table_",lowercase(randstring(5))))
    
    n = size(m, 1)
    d = [Dict() for i in 1:n]
    for i in 1:n
        for (col, colname) in enumerate(colnames)
            colname = replace(replace(colname,"[",""),"]","")
            d[i][colname] = m[i,col]
        end
    end        
    VegaData(name = name, values = d)
end

Vega.VegaData

In [4]:
using Vega

import Vega: VegaData

# Construct VegaData from a given Markov Chain
VegaData(chains::Mamba.AbstractChains, args...) = VegaData(squeeze(chains.value, 3), chains.names, name="samples")

function VegaData{S<:AbstractString}(m::Matrix, colnames::Vector{S}; 
    name::AbstractString = string("table_",lowercase(randstring(5))))
    
    n = size(m, 1)
    d = [Dict() for i in 1:n]
    for i in 1:n
        for (col, colname) in enumerate(colnames)
            colname = replace(replace(colname,"[",""),"]","")
            d[i][colname] = m[i,col]
        end
    end        
    VegaData(name = name, values = d)
end


function scales(param::AbstractString) 
    xscale = VegaScale(name="x", range="width", 
        domain=VegaDataRef(data=param, field="bin_$param"))
    yscale = VegaScale(name="y", range="height",
        domain=VegaDataRef(data=param, field="count"))
    [xscale, yscale]
end

parammarks(param) =
    VegaMark(_type="rect", 
        from=VegaMarkFrom(data=param), 
        properties=VegaMarkProperties(
            update=VegaMarkPropertySet(
                x=    VegaValueRef(scale = "x", field = "bin_$param"),
                width=VegaValueRef(value = 3.5),
                y=    VegaValueRef(scale = "y", field = "count"),
                y2=   VegaValueRef(scale = "y", value=0),
                fill= VegaValueRef(value = "#000"))))


paramgroup(param::AbstractString) = 
    VegaMark(_type="group", scales=scales(param), marks=[parammarks(param)], 
        properties=VegaMarkProperties(
            update=VegaMarkPropertySet(
                x =      VegaValueRef(value = 0),
                y =      VegaValueRef(value = 0),
                width =  VegaValueRef(value = 450),
                height = VegaValueRef(value = 100))))


# transformation 
bin_transform(param) =
    VegaTransform(Dict(
        "type" => "bin",
        "maxbins" => 40,
        "field" => param, 
        "output" => Dict(
            "start" => "bin_$param")))

countdata(param) = 
    VegaData(
        name=param, 
        source="samples", 
        transform=[count_transform("bin_$param")])

count_transform(groupby) = 
    VegaTransform(Dict(
        "type" => "aggregate",
        "groupby" => [groupby],
        "summarize" => Dict("*"=> ["count"])))


function plotit(sim)
    data = VegaData(sim)

    v = VegaVisualization()
    v.data = [data]
    
    data.transform = VegaTransform[]
    v.marks = VegaMark[]

    for (i,param) in enumerate(keys(data.values[1]))
        
        group = paramgroup(param)
        group.properties.update.y.value = (i-1) * 100
        
        push!(data.transform, bin_transform(param))
        push!(v.data,         countdata(param))
        push!(v.marks,        group)
    end
    v
end

plotit(sim[:, 1:8, 1])

In [5]:
using Vega
import KernelDensity.kde

function plot_species(c::Mamba.AbstractChains, kde=false)
    @assert size(c,[2,3]) == (1,1)
    
    data = c.value|>vec
    
    v = kde ? plot_kde(data) : histogram(x=data, nbins = 40)
    
    title!(v, title=c.names[1])
end

function plot_kde(data::Vector)
    npoints = 2^14
    bw = maximum(data) / npoints * 5
    k  = kde(data, npoints=npoints, bandwidth=bw)
    
    v=lineplot(x=k.x,y=k.density)
    v.scales[1].zero=false
    v
end

function slices(a,dim)
    inds = Any[(:) for i in 1:length(size(a))]
    map(1:size(a,dim)) do i
        inds[dim] = i
        getindex(a,inds...)
    end
end

function plot(c::Mamba.AbstractChains)
    reduce(layer,map(plot_species,slices(c,2)))
end

@time v = plot_species(sim[:,8,1])

In [6]:
# snippet for easy field-value filtering

import Vega: AbstractVegaType
import Base: getindex

function getindex(c::Vector{VegaScale}, kv::Pair)
    k,v = kv
    i=find(x->getfield(x, k) == v, c)
    c[i]
end

v.scales[:name=>"x"]

5.007562 seconds (2.86 M allocations: 115.183 MB, 2.69% gc time)


1-element Array{Vega.VegaScale,1}:
 Vega.VegaScale("x","ordinal",Vega.VegaDataRef("table_ozfo8","x",nothing,true),nothing,nothing,"width",nothing,nothing,nothing,nothing,nothing,nothing,nothing,nothing,nothing,nothing,nothing,nothing,nothing)